Name: Pruthvik Kakadiya

Movie Recommendation Using Matrix Factorization 

In [104]:
import pandas as pd
import numpy as np

In [105]:
#Reading only first few rows of ratings database as the database is too big

In [106]:
ratings = pd.read_csv("ml-20m/ratings.csv", skipfooter = 19994500, engine = 'python')

In [107]:
ratings

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
...,...,...,...,...
5758,54,2013,4.0,975440957
5759,54,2015,4.0,974906025
5760,54,2021,1.0,974836589
5761,54,2026,2.0,974839538


In [108]:
#now lets check how many unique users are in the dataframe we have read

In [109]:
ratings['userId'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54], dtype=int64)

In [110]:
'''
now we have to convert the ratings database into 
a matrix of userID*moviesID where the perticular 
cell will have the movie rating given by that user.
'''

'\nnow we have to convert the ratings database into \na matrix of userID*moviesID where the perticular \ncell will have the movie rating given by that user.\n'

In [111]:
ratings.userId = ratings.userId.astype('category').cat.codes.values
ratings.movieId = ratings.movieId.astype('category').cat.codes.values

In [112]:
index=list(ratings['userId'].unique())
columns=list(ratings['movieId'].unique())
index=sorted(index)
columns=sorted(columns)
df=pd.pivot_table(data=ratings,values='rating',index='userId',columns='movieId').fillna(0)

In [113]:
df.head(10)

movieId,0,1,2,3,4,5,6,7,8,9,...,2263,2264,2265,2266,2267,2268,2269,2270,2271,2272
userId,,,,,,,,,,,,,,,,,,,,,
0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [114]:
#the code from assignment 1 fot matrix factorization using gredient decent.

In [115]:
import progressbar as pb
def matrix_factorization(Matrix, P, Q, K, iterations=1000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in pb.progressbar(range(iterations)):
        for i in range(len(Matrix)):
            for j in range(len(Matrix[i])):
                if Matrix[i][j] > 0:
                    element_ij = Matrix[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * element_ij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * element_ij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        error = 0
        for i in range(len(Matrix)):
            for j in range(len(Matrix[i])):
                if Matrix[i][j] > 0:
                    error = error + pow(Matrix[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        error = error + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        if error < 0.001:
            break
    print("Matrix Factorization Completed")
    return P, Q.T

In [116]:
import time
start_time = time.time()
ratings_array = np.array(df)

In [117]:
N = len(ratings_array)
M= len(ratings_array[0])


In [118]:
K = 2
P = np.random.rand(N,K)
Q = np.random.rand(M,K)

In [119]:
'''
now lets run the matrix_factorization on the new 
userID*moviesID matrix to predict the unrated movies.
'''

'\nnow lets run the matrix_factorization on the new \nuserID*moviesID matrix to predict the unrated movies.\n'

In [120]:
new_P, new_Q = matrix_factorization(ratings_array, P, Q, K)
new_prediction_matrix = np.dot(new_P, new_Q.T)

100% (1000 of 1000) |####################| Elapsed Time: 0:05:34 Time:  0:05:34


Matrix Factorization Completed


In [121]:
#lets save the prediction matrix in a csv file to be used later

In [122]:
pd.DataFrame(new_prediction_matrix).to_csv('predictions.csv')

In [123]:
pred = pd.read_csv('predictions.csv')
movies = pd.read_csv("ml-20m/movies.csv")

In [124]:
#lets create a function to generate movies recommendations based on the predction matrix and user ID

In [125]:
def get_recommendation (df, user, n=10):
  a = df.to_numpy()[0][user:]
  l = sorted(range(len(a)), key=lambda i: a[i])[-1*n:]
  movies_list = []
  for i in l:
    movies_list.append([movies['title'][i], movies['genres'][i]])
  return movies_list

In [126]:
#lets run the movies recommendation function for user 1 to recommend 15 movies.

In [127]:
get_recommendation(df= pred, user= 1, n= 15)

[['Godfather: Part II, The (1974)', 'Crime|Drama'],
 ['Some Folks Call It a Sling Blade (1993)', 'Drama|Thriller'],
 ['Everest (1998)', 'Documentary|IMAX'],
 ['Endless Summer 2, The (1994)', 'Adventure|Documentary'],
 ['Married to the Mob (1988)', 'Comedy'],
 ['Little Mermaid, The (1989)', 'Animation|Children|Comedy|Musical|Romance'],
 ['Vampires (1998)', 'Horror|Western'],
 ['Ill Gotten Gains (1997)', 'Drama'],
 ['Star Kid (1997)', 'Adventure|Children|Fantasy|Sci-Fi'],
 ['Celebrity (1998)', 'Comedy'],
 ['Frenzy (1972)', 'Thriller'],
 ['Booty Call (1997)', 'Comedy|Romance'],
 ['Air Force One (1997)', 'Action|Thriller'],
 ['Citizen Ruth (1996)', 'Comedy|Drama'],
 ['Escape to Witch Mountain (1975)', 'Adventure|Children|Fantasy']]

In [128]:
#lets ask user to enter userID and how many movies to recommend.

In [129]:
user = int(input("Enter UserId:"))
count = int(input("How many movies to recommend:"))

Enter UserId:3
How many movies to recommend:15


In [130]:
#lets run the movie recommendation function for user given data.

In [133]:
print("Recommendation from non-rated movies.")
recommendation = get_recommendation(df= pred, user= user, n= count)
for s in recommendation:
    print(*s)
print("Execution Time: %s seconds" % (time.time() - start_time))

Recommendation from non-rated movies.
Psycho (1960) Crime|Horror
Hostile Intentions (1994) Action|Drama|Thriller
Callejón de los milagros, El (1995) Drama
Dazed and Confused (1993) Comedy
Working Girl (1988) Comedy|Drama|Romance
Kidnapped (1960) Adventure|Children|Drama
Shattered Image (1998) Drama|Thriller
Tar (1996) Drama
Dark City (1998) Adventure|Film-Noir|Sci-Fi|Thriller
Rugrats Movie, The (1998) Animation|Children|Comedy
Rope (1948) Crime|Drama|Thriller
Donnie Brasco (1997) Crime|Drama
Excess Baggage (1997) Adventure|Romance
My Fellow Americans (1996) Comedy
Apple Dumpling Gang, The (1975) Children|Comedy|Western
Execution Time: 764.3443098068237 seconds
